In [1]:
import warnings
warnings.filterwarnings('ignore')

## 使用
我们提供了一键运行脚本，由于使用了多线程，并不支持jupyter中运行，
### 如何运行
- 安装依赖
```
pip install dreamsboard["vector"] -U
```

我们对每个脚本提供了一些环境变量，除了基本的推理服务环境之外，还有一些资源配置的环境变量
- 服务商环境
```

export DEEPSEEK_API_BASE="https://api.deepseek.com/v1"
export DEEPSEEK_API_MODEL="deepseek-chat"
export DEEPSEEK_API_KEY="sk-api"
export ZHIPUAI_API_BASE="https://open.bigmodel.cn/api/paas/v4"
export ZHIPUAI_API_MODEL="glm-4-plus"
export ZHIPUAI_API_KEY="api.key"

```

- 资源配置
```
# rerank的模块，需要支持 from sentence_transformers import CrossEncoder
export cross_encoder_path="/mnt/ceph/develop/jiawei/model_checkpoint/jina-reranker-v2-base-multilingual"
# embedding的模块，需要支持 from sentence_transformers import SentenceTransformer
export embed_model_path="/mnt/ceph/develop/jiawei/model_checkpoint/m3e-base"
# 任务描述
export start_task_context="大模型MLA技术细节"
# 是否是一个新任务
export allow_init="true"
```


导入环境后，请使用如下脚本`test_task/glm/main.py`运行你需要的服务

- 推理
```
python test_task/glm/main.py
```
> 这个脚本会在执行位置创建本地目录，包含了`storage`中间过程，`vector_store`矢量库

> 这个过程会涉及大量的io处理请使用本地磁盘，网络磁盘会影响调度速度



 
### 渲染文档

我们也提供了一个默认的文档渲染封装，如果你想渲染其它形式的结构，请读取`storage`中间过程自行编写代码

```
python test_task/glm/printmd.md
```
> 脚本会读取`start_task_context`环境变量

 

### 任务表格

In [7]:
from dreamsboard.engine.storage.task_step_store.simple_task_step_store import SimpleTaskStepStore

from dreamsboard.dreams.task_step_to_question_chain.weaviate.prepare_load import get_query_hash
import os
from dreamsboard.document_loaders.structured_storyboard_loader import StructuredStoryboard
start_task_context="大模型MLA技术细节"
base_path = f'./{get_query_hash(start_task_context)}/'
store_load = SimpleTaskStepStore.from_persist_dir(persist_dir=f'./{base_path}/storage')
 
structured_storyboard = StructuredStoryboard(json_data=[step.__dict__ for step in list(store_load.task_step_all.values())])

structured_storyboard.parse_table()

,task_step_id,shot_number,scene_number,start_task_context,aemo_representation_context,task_step_name,task_step_description,task_step_level,task_step_question,task_step_question_context,task_step_question_answer,ref_task_step_id
0,53cf56ba-f517-4156-8bcf-942921aee390,1,story_board0,大模型MLA技术细节,### Step 1: 分析近几年研究领域的技术框架与方法论\n\n**研究成果**:\n1...,分析近几年研究领域的技术框架与方法论,深度学习框架：卷积神经网络 (CNN) 在图像识别和视频分析中广泛应用，如ResNet、Ef...,0,### 问题\n\n在分析近几年研究领域的技术框架与方法论时，深度学习框架如卷积神经网络 (...,"[{'ref_id': '454918900731096864', 'chunk_id': ...",在医疗影像分析中，混合模型通过结合CNN和Transformer的优势，能够更准确地识别病变...,
1,a9901553-4ed2-4163-9b05-360018088c92,2,story_board1,大模型MLA技术细节,### Step 1: 分析近几年研究领域的技术框架与方法论\n\n**研究成果**:\n1...,研究论文中采用的主要框架在不同任务中的应用与变体,CNN变体：EfficientNet通过复合缩放方法提升效率和性能。MobileNet适用于...,1,### 问题\n\n在“研究论文中采用的主要框架在不同任务中的应用与变体”这一任务步骤中，*...,"[{'ref_id': '454846687148335366', 'chunk_id': ...",在医疗影像分析中，跨模态注意力机制可以结合动态计算，通过自适应推理策略优化影像与文本数据的交...,
2,2cd5432a-efc7-4ca2-9340-0439ece15a4e,3,story_board2,大模型MLA技术细节,### Step 1: 分析近几年研究领域的技术框架与方法论\n\n**研究成果**:\n1...,评估学术界的技术进步与局限性,性能提升：精度提高如BERT在多项NLP任务中达到SOTA。效率提升如EfficientNe...,2,### 问题\n\n在评估学术界的技术进步与局限性时，如何有效平衡模型性能的提升（如精度和效...,"[{'ref_id': '454959888553742592', 'chunk_id': ...",在医疗领域，BERT模型在电子健康记录（EHR）分析中的表现尤为突出，通过预训练和微调，BE...,
3,f9aaddfd-317d-401a-9b0d-5183b2bb866c,4,story_board3,大模型MLA技术细节,### Step 1: 分析近几年研究领域的技术框架与方法论\n\n**研究成果**:\n1...,探讨计算模型在不同数据集与应用场景下的适用性与泛化能力,多领域应用：多模态学习如CLIP结合图像和文本进行联合学习。跨领域迁移学习如在源领域预训练，...,3,### 问题\n\n在探讨计算模型在不同数据集与应用场景下的适用性与泛化能力时，如何有效利用...,"[{'ref_id': '454846660891728744', 'chunk_id': ...",在医疗影像分析中，多模态学习可以通过结合图像和文本数据，提升模型对复杂病例的诊断能力。例如，...,
4,07d08b9c-62e7-47f6-aae2-a814b7a455cc,5,story_board4,大模型MLA技术细节,### Step 1: 分析近几年研究领域的技术框架与方法论\n\n**研究成果**:\n1...,分析最新算法的稳定性与容错性,稳定性优化：对抗鲁棒性如PGD对抗训练。模型正则化如Dropout、Weight Decay...,4,### 问题\n\n在分析最新算法的稳定性与容错性时，如何通过引入对抗样本进行训练（如PGD...,"[{'ref_id': '454847291425149688', 'chunk_id': ...",在模型正则化方面，Dropout和Weight Decay等技术在提升模型稳定性和容错性方面...,
5,262e3c6f-63f3-4ead-abf9-cd9cd4a81c98,6,story_board5,大模型MLA技术细节,### Step 1: 分析近几年研究领域的技术框架与方法论\n\n**研究成果**:\n1...,评估论文中提出的未来研究方向与挑战,新研究问题：可解释性提高模型透明度和可解释性。隐私保护如差分隐私在机器学习中的应用。挑战：数...,5,### 问题\n\n在评估论文中提出的未来研究方向与挑战时，如何平衡模型的可解释性与隐私保护...,"[{'ref_id': '454984255704670602', 'chunk_id': ...",在探讨未来研究方向与挑战时，模型的可解释性与隐私保护之间的平衡是一个关键问题。特别是在资源受...,


### 渲染效果

In [8]:

from dreamsboard.dreams.task_step_md.base import TaskStepMD

from IPython.display import Markdown, display
 
task_step_store = SimpleTaskStepStore.from_persist_dir(f'./{base_path}/storage')
task_step_md = TaskStepMD(task_step_store)
md_text =   task_step_md.format_md()

display(Markdown(md_text.text))


# 大模型MLA技术细节 


### 分析近几年研究领域的技术框架与方法论 [task_id](53cf56ba-f517-4156-8bcf-942921aee390)<sup>0</sup>

在医疗影像分析中，混合模型通过结合CNN和Transformer的优势，能够更准确地识别病变区域。例如，在肺部CT影像分析中，CNN用于提取局部特征，而Transformer则用于建模全局依赖关系，从而提高了诊断的准确性。在自动驾驶领域，强化学习框架通过模拟环境中的复杂决策过程，能够有效提升车辆的自主驾驶能力。例如，通过PPO算法训练的自动驾驶模型在城市交通场景中表现出色，能够应对复杂的交通状况。自监督学习技术在减少标注数据需求方面也取得了显著进展。例如，SimCLR在无监督图像分类任务中通过对比学习显著提升了模型的性能，而BYOL则通过自监督学习在半监督文本分类任务中表现出色。未来研究方向中，少样本学习和联邦学习等技术为解决数据质量和计算资源问题提供了新的思路。例如，少样本学习通过利用少量标注数据训练模型，能够在数据稀缺的场景中取得良好的效果，而联邦学习则通过分布式训练保护数据隐私，同时提升模型的泛化能力。

### 研究论文中采用的主要框架在不同任务中的应用与变体 [task_id](a9901553-4ed2-4163-9b05-360018088c92)<sup>1</sup>

在医疗影像分析中，跨模态注意力机制可以结合动态计算，通过自适应推理策略优化影像与文本数据的交互，提升诊断准确性。在自动驾驶领域，结合自监督学习与元学习的方法，可以在少样本场景下快速适应新的驾驶环境，提升模型的泛化能力。轻量化设计中的核方法可以在车载嵌入式系统中实现高效计算，通过近似注意力矩阵降低计算复杂度，确保实时性。此外，理论分析可以进一步探讨这些混合模型在不同领域中的性能表现，为实际应用提供理论支持。

### 评估学术界的技术进步与局限性 [task_id](2cd5432a-efc7-4ca2-9340-0439ece15a4e)<sup>2</sup>

在医疗领域，BERT模型在电子健康记录（EHR）分析中的表现尤为突出，通过预训练和微调，BERT能够有效提取病历中的关键信息，提升诊断准确性。EfficientNet在资源优化方面的具体数据表明，其在ImageNet数据集上的表现优于传统卷积神经网络，同时显著减少了计算资源的消耗。在金融领域，深度学习模型在欺诈检测中的应用也取得了显著成效，通过结合时间序列分析和异常检测技术，模型能够实时识别潜在的欺诈行为。在自动驾驶领域，结合自监督学习与元学习的方法，可以在少样本场景下快速适应新的驾驶环境，提升模型的泛化能力。此外，理论分析与实验验证的结合，如通过分布鲁棒优化和方差正则化技术，为模型提供了理论上的保证，确保其在实际应用中的鲁棒性和可靠性。

### 探讨计算模型在不同数据集与应用场景下的适用性与泛化能力 [task_id](f9aaddfd-317d-401a-9b0d-5183b2bb866c)<sup>3</sup>

在医疗影像分析中，多模态学习可以通过结合图像和文本数据，提升模型对复杂病例的诊断能力。例如，利用CLIP模型进行图像和文本的联合学习，能够捕捉到图像中的关键特征与文本描述之间的语义关联，从而提升模型的解释性和诊断准确性。跨领域迁移学习则可以通过模型重编程技术，将预训练模型从视觉领域迁移到医疗领域，无需大量标注数据即可适应新任务。对抗训练和域自适应技术可以进一步增强模型在面对不同医疗数据集时的鲁棒性，确保其在各种临床场景下的稳定表现。通过研究泛化理论，设计更高效的优化算法，可以进一步提升模型在跨领域任务中的泛化能力，确保其在面对新数据时保持高性能。

### 分析最新算法的稳定性与容错性 [task_id](07d08b9c-62e7-47f6-aae2-a814b7a455cc)<sup>4</sup>

在模型正则化方面，Dropout和Weight Decay等技术在提升模型稳定性和容错性方面也发挥了重要作用。Dropout通过在训练过程中随机丢弃部分神经元，防止模型过拟合，从而提升模型的泛化能力。Weight Decay则通过在损失函数中加入正则化项，限制模型参数的过大增长，进一步防止过拟合。这些正则化技术在实际应用中能够有效提升模型的稳定性和容化能力，尤其是在面对复杂数据分布和噪声时。

此外，孤立森林和One-Class SVM等异常检测技术在分布式训练中的应用也值得探讨。孤立森林通过构建随机树来隔离异常样本，而One-Class SVM则通过构建超平面来区分正常样本与异常样本。这些方法在联邦学习中的应用可以有效提升模型的容错性，尤其是在面对恶意客户端攻击时。通过结合这些异常检测技术，可以进一步增强模型在面对异常数据时的稳定性和鲁棒性。

在实际应用案例中，这些方法在复杂和动态环境下的表现也值得关注。例如，在金融欺诈检测中，PGD对抗训练和联邦学习中的容错机制能够有效提升模型在面对对抗攻击时的鲁棒性，同时保护用户数据的隐私。在医疗诊断中，模型正则化技术和异常检测方法能够帮助模型在面对噪声数据时保持较高的准确性。这些实际案例进一步验证了这些方法在提升模型稳定性和容错性方面的有效性。

### 评估论文中提出的未来研究方向与挑战 [task_id](262e3c6f-63f3-4ead-abf9-cd9cd4a81c98)<sup>5</sup>

在探讨未来研究方向与挑战时，模型的可解释性与隐私保护之间的平衡是一个关键问题。特别是在资源受限的环境中，如何在不牺牲模型性能的前提下，确保数据的隐私和模型的可解释性，是一个亟待解决的难题。例如，在医疗领域，如何在保护患者隐私的同时，提供透明的诊断模型，是一个重要的研究方向。通过结合联邦学习、差分隐私和可解释性增强技术，可以在保护隐私的同时，提供透明且高效的AI解决方案。

此外，数据质量与计算资源限制也是未来研究的重要挑战。在低质量数据环境下，如何通过数据清洗和增强技术提高数据质量，同时结合差分隐私技术保护数据隐私，是一个值得深入探讨的问题。在资源受限的环境中，如何通过梯度压缩和通信优化技术，减少计算资源的消耗，提高模型的训练效率，也是一个重要的研究方向。

结合量子计算和边缘计算等新兴技术，可以进一步提升模型的性能和隐私保护能力。例如，利用量子计算加速联邦学习中的梯度计算，或在边缘设备上实现高效的差分隐私保护，都是具有前瞻性的研究方向。这些技术的结合，不仅可以在资源受限的环境中有效应用，还能提升用户对AI系统的信任，促进AI在复杂环境中的广泛应用。




# References  

[0] ,chunk_id:454918900731096864 
[0] ,chunk_id:454849250644648612 
[0] ,chunk_id:454848644537032228 
[1] ,chunk_id:454846687148335366 
[1] ,chunk_id:454847026521023802 
[1] ,chunk_id:454849056084511186 
[2] ,chunk_id:454959888553742592 
[2] ,chunk_id:454848129553553028 
[2] ,chunk_id:454846731731167836 
[3] ,chunk_id:454846660891728744 
[3] ,chunk_id:454895302186370932 
[3] ,chunk_id:454984230906632244 
[4] ,chunk_id:454847291425149688 
[4] ,chunk_id:454847099910337174 
[4] ,chunk_id:454959876536011888 
[5] ,chunk_id:454984255704670602 
[5] ,chunk_id:454845858389724938 
[5] ,chunk_id:454845753539980440 

